In [16]:
import pandas as pd
import os
from glob import glob

In [17]:
def read_file(filename):
    df = pd.read_csv(filename)
    df["sleep"] = df["stages"] > 0
    df["linetime"] = pd.to_datetime(df["linetime"])
    return df

In [18]:
def generate_slide_wins(df, winsize=11):
    
    seq_id = 0
    transformed_df = []
    list_of_indexes=[] 
    labels_5min = []
    labels_0min = []
    linetime = []
    activity = []
    mean_hr = []
    
    df.index.to_series().rolling(winsize, center=True).apply((lambda x: list_of_indexes.append(x.tolist()) or 0), raw=False)
    
    
    for idx in list_of_indexes:
        # to Do: this column name "ground truth" shuld be changed later
        labels_5min.append(df.iloc[idx].iloc[winsize//2]["ground_truth_5min"]) #take the middle value of the window 
        labels_0min.append(df.iloc[idx].iloc[winsize//2]["ground_truth_0min"]) #take the middle value of the window
        linetime.append(df.iloc[idx].iloc[winsize//2]["linetime"])   #we need this for sleep metrics calculation later
        activity.append(df.iloc[idx].iloc[winsize//2]["activity"])   #we need this for sleep metrics calculation later
        mean_hr.append(df.iloc[idx].iloc[winsize//2]["mean_hr"])
        
        tmp_df = df.iloc[idx].copy()
        tmp_df["seq_id"] = seq_id
        seq_id += 1
        transformed_df.append(tmp_df)

    return pd.concat(transformed_df), pd.Series(labels_5min), pd.Series(labels_0min),pd.Series(linetime), pd.Series(activity), pd.Series(mean_hr)



In [ ]:
input_files = glob("../data/Processed_Mesa_gt_WithandWithout_tolerance/*.csv.gz")
all_ys_5min = []
all_ys_0min = []
all_ids = []
all_linetime = []
all_activity = []
all_mean_hr = []


# Read files and roll 21 windows
for file in input_files:
    df = read_file(file)
#     print("find Nans",df.isnull().sum())
    transformed_df, labels_5min, labels_0min, linetime, activity, mean_hr = generate_slide_wins(df, 21)

    
    ids = pd.Series(labels_5min.shape[0]*[df["mesaid"].unique()[0]])
    
    all_ids.append(ids)
    all_ys_5min.append(labels_5min)
    all_ys_0min.append(labels_0min)
    all_linetime.append(linetime)
    all_activity.append(activity)
    all_mean_hr.append(mean_hr)




    

In [ ]:
all_ids = pd.concat(all_ids)
all_ys_5min = pd.concat(all_ys_5min)
all_ys_0min = pd.concat(all_ys_0min)
all_linetime = pd.concat(all_linetime)
# all_features = pd.concat(all_features)
all_activity = pd.concat(all_activity)
all_mean_hr = pd.concat(all_mean_hr)

# give name to columns 
all_ids = pd.DataFrame(all_ids)
all_ids.columns = ['id']

all_ys_5min = pd.DataFrame(all_ys_5min)
all_ys_5min.columns = ['label_5min']

all_ys_0min = pd.DataFrame(all_ys_0min)
all_ys_0min.columns = ['label_0min']

all_linetime = pd.DataFrame(all_linetime)
all_linetime.columns = ['linetime']

all_activity = pd.DataFrame(all_activity)
all_activity.columns = ['activity']

all_mean_hr = pd.DataFrame(all_mean_hr)
all_mean_hr.columns = ['mean_hr']

In [14]:
all_data = pd.concat([all_ids.reset_index(drop=True), all_ys_5min.reset_index(drop=True),
                      all_ys_0min.reset_index(drop=True), all_activity.reset_index(drop=True), 
                      all_mean_hr.reset_index(drop=True),all_linetime.reset_index(drop=True)], axis=1)
all_data

NameError: name 'all_ids' is not defined

In [ ]:
# to prevent from memory crash
del all_mean_hr, all_activity, all_linetime, all_ys_0min, all_ys_5min, all_ids

In [7]:
# list of test/val/train ids
test_ids = [6029, 5821, 2946, 4794,  347, 5177, 3086, 5570, 3771, 3389, 5899,
        800, 5792,   70, 4026,  838, 3874, 1085, 5219, 5897, 6416, 1089,
       5659, 4521,  565, 5151, 6723, 2903, 6320, 4649, 2548, 1300, 4088,
       5608, 5393, 6739, 4333,  457, 5632, 6052, 2602, 5665, 1650, 2966,
       2933, 2638, 5002, 3303, 1766, 5983, 2317,  140, 4815, 4285, 3625,
       5304, 4873, 2467, 4617, 2416, 2615, 1759, 5131, 2598, 5748,  225,
       2723, 5284,  893, 6730, 1077, 4575, 3216, 3547, 3275, 5754, 5166,
       5557, 2687,  860, 5884, 1309,  460, 5596, 5292, 2388, 2150,  728,
       4140, 5326, 6317, 5414, 4294, 4168, 3072, 4015, 4820, 2549, 6501,
       6790, 3717, 1694,  118, 3750, 4580, 2894, 5451, 3980, 1055, 4461,
        692]

val_ids = [2487, 6280, 1074,  111, 4860, 3016, 3647,  863, 6091,   48, 5429,
       1087, 4839, 2507, 3018, 2303, 5364, 3921, 3591, 3833, 3987, 6027,
       3537, 5876, 3413, 5440, 6367, 2807,  318,  534, 2868, 6656, 5381,
       2728, 1707, 1900, 1733, 4186, 5677, 3239, 1717, 4782,  386, 3585,
       5788, 2987, 2217, 3293, 4671, 5793, 4849,  862, 4496, 3822, 4484,
       1098, 1989, 5500,  499, 3133,  837, 6440, 2668, 2764, 1964, 1043,
       1907, 6653, 3089, 6622, 4432, 6180, 5888, 3920, 3639, 4111, 4928,
       2414, 4044,  632, 5838, 1391, 4998, 3516, 2050, 1102, 4875, 3290,
        852,  771, 3936, 1478, 2218,  626, 2463, 3873, 1531,  602, 5532,
       1256, 5789, 5623, 5011, 3441,  196,  501, 4330,  558,  458, 2123,
       2904]
# train_ids = []



test_data = []
train_data = []
val_data = []
for i in range(len(all_data)):
    if(all_data.loc[i, 'id'] in test_ids):
        test_data.append(all_data[ :i ])
    elif(all_data.loc[i, 'id'] in val_ids):
        val_data.append(all_data[ :i ])
    else:
        train_data.append(all_data[ :i ])
        
testset = pd.concat(test_data) 
trainset = pd.concat(train_data) 
valset = pd.concat(val_data)

#define testset
X_test = testset[['id', 'activity', 'mean_hr', 'linetime']]
y_test_5min = testset[['label_5min']]
y_test_0min = testset[['label_0min']]

#define trainset
X_train = trainset[['id', 'activity', 'mean_hr', 'linetime']]
y_train_5min = trainset[['label_5min']]
y_train_0min = trainset[['label_0min']]

#define valset
X_val = valset[['id', 'activity', 'mean_hr', 'linetime']]
y_val_5min = valset[['label_5min']]
y_val_0min = valset[['label_0min']]



In [ ]:
#PLEASE SAVE THESE DATASETS AND SHARE THEM WITH ME